# Notebook 02 — Exploratory Data Analysis

## Credit Risk and Dollarization in Cambodia: Dual-Currency Spread Analysis

This notebook performs comprehensive exploratory analysis of the USD and KHR interest rate spreads (term loan rate − term deposit rate) computed in Notebook 01.

**Contents:**
1. Descriptive Statistics (full sample & sub-period)
2. Normality Tests (Shapiro-Wilk, Jarque-Bera)
3. Stationarity Tests (Augmented Dickey-Fuller)
4. Autocorrelation Analysis (ACF/PACF)
5. Correlation Analysis
6. Publication-Quality Visualizations (Figures 1–6)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle
from scipy import stats
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings('ignore')

# Publication-quality plot settings
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'legend.fontsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'font.family': 'serif'
})

print('Libraries loaded successfully.')

In [ ]:
# ─── Load Data ───────────────────────────────────────────────────────────────
usd = pd.read_csv('../data/processed/spreads_usd_new_amount.csv', parse_dates=['date'], index_col='date')
khr = pd.read_csv('../data/processed/spreads_khr_new_amount.csv', parse_dates=['date'], index_col='date')
rates = pd.read_csv('../data/processed/all_rates_wide_new_amount.csv', parse_dates=['Date'], index_col='Date')

# Combine spreads into a single DataFrame
spreads = pd.DataFrame({
    'USD_Spread': usd['spread'],
    'KHR_Spread': khr['spread']
})

print(f'Sample period: {spreads.index[0].strftime("%b %Y")} – {spreads.index[-1].strftime("%b %Y")}')
print(f'Total observations: {len(spreads)}')
spreads.head()

---
## 1. Descriptive Statistics

In [ ]:
# ─── Full-Sample Descriptive Statistics ──────────────────────────────────────
def descriptive_stats(series, name):
    """Compute comprehensive descriptive statistics for a spread series."""
    return pd.Series({
        'Mean (%)': series.mean(),
        'Std. Dev. (%)': series.std(),
        'Min (%)': series.min(),
        'Max (%)': series.max(),
        'Median (%)': series.median(),
        'Skewness': series.skew(),
        'Kurtosis': series.kurtosis(),
        'P5 (%)': series.quantile(0.05),
        'P25 (%)': series.quantile(0.25),
        'P50 (%)': series.quantile(0.50),
        'P75 (%)': series.quantile(0.75),
        'P95 (%)': series.quantile(0.95),
        'N': len(series)
    }, name=name)

stats_table = pd.DataFrame([
    descriptive_stats(spreads['USD_Spread'], 'USD Spread'),
    descriptive_stats(spreads['KHR_Spread'], 'KHR Spread')
]).T

print('\n══════════════════════════════════════════════════════════════')
print('          TABLE 1: Descriptive Statistics — Full Sample')
print('══════════════════════════════════════════════════════════════')
print(stats_table.round(4).to_string())
print('══════════════════════════════════════════════════════════════')

In [ ]:
# ─── Sub-Period Descriptive Statistics ────────────────────────────────────────
periods = {
    'Pre-COVID (2013–2019)': ('2013-01-01', '2019-12-31'),
    'COVID (2020–2021)':     ('2020-01-01', '2021-12-31'),
    'Post-COVID (2022–2025)':('2022-01-01', '2025-12-31')
}

sub_stats = []
for pname, (start, end) in periods.items():
    mask = (spreads.index >= start) & (spreads.index <= end)
    sub = spreads[mask]
    row = {
        'Period': pname,
        'N': len(sub),
        'USD Mean (%)': sub['USD_Spread'].mean(),
        'USD Std (%)': sub['USD_Spread'].std(),
        'KHR Mean (%)': sub['KHR_Spread'].mean(),
        'KHR Std (%)': sub['KHR_Spread'].std(),
        'Correlation': sub['USD_Spread'].corr(sub['KHR_Spread'])
    }
    sub_stats.append(row)

sub_df = pd.DataFrame(sub_stats).set_index('Period')
print('\n═══════════════════════════════════════════════════════════════════════')
print('         TABLE 1b: Sub-Period Summary Statistics')
print('═══════════════════════════════════════════════════════════════════════')
print(sub_df.round(4).to_string())
print('═══════════════════════════════════════════════════════════════════════')

---
## 2. Normality Tests

In [ ]:
# ─── Normality Tests ─────────────────────────────────────────────────────────
normality_results = []
for col, label in [('USD_Spread', 'USD Spread'), ('KHR_Spread', 'KHR Spread')]:
    sw_stat, sw_p = stats.shapiro(spreads[col])
    jb_stat, jb_p = stats.jarque_bera(spreads[col])
    normality_results.append({
        'Series': label,
        'Shapiro-Wilk Stat': sw_stat,
        'Shapiro-Wilk p-value': sw_p,
        'Jarque-Bera Stat': jb_stat,
        'Jarque-Bera p-value': jb_p,
        'Normal at 5%?': 'Yes' if (sw_p > 0.05 and jb_p > 0.05) else 'No'
    })

norm_df = pd.DataFrame(normality_results).set_index('Series')
print('\n═══════════════════════════════════════════════════════════════')
print('                    Normality Test Results')
print('═══════════════════════════════════════════════════════════════')
print(norm_df.round(4).to_string())
print('═══════════════════════════════════════════════════════════════')
print('\nInterpretation: If p-value < 0.05, we reject the null hypothesis')
print('of normality. This is common for financial data and justifies')
print('the use of non-parametric robustness checks.')

---
## 3. Stationarity Tests (ADF)

In [ ]:
# ─── Augmented Dickey-Fuller Tests ───────────────────────────────────────────
adf_results = []
for col, label in [('USD_Spread', 'USD Spread'), ('KHR_Spread', 'KHR Spread')]:
    result = adfuller(spreads[col], autolag='AIC')
    adf_results.append({
        'Series': label,
        'ADF Statistic': result[0],
        'p-value': result[1],
        'Lags Used': result[2],
        'Critical 1%': result[4]['1%'],
        'Critical 5%': result[4]['5%'],
        'Critical 10%': result[4]['10%'],
        'Stationary at 5%?': 'Yes' if result[1] < 0.05 else 'No'
    })

adf_df = pd.DataFrame(adf_results).set_index('Series')
print('\n═══════════════════════════════════════════════════════════════════')
print('              Augmented Dickey-Fuller Test Results')
print('═══════════════════════════════════════════════════════════════════')
print(adf_df.round(4).to_string())
print('═══════════════════════════════════════════════════════════════════')
print('\nInterpretation: If p-value < 0.05, we reject the null hypothesis')
print('of a unit root → the spread is stationary. Stationarity supports')
print('the mean-reverting OU model assumption.')

---
## 4. Autocorrelation Analysis

In [ ]:
# ─── ACF / PACF Plots ────────────────────────────────────────────────────────
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

plot_acf(spreads['USD_Spread'], lags=30, ax=axes[0, 0], color='#2196F3', alpha=0.05)
axes[0, 0].set_title('ACF — USD Spread', fontweight='bold')

plot_pacf(spreads['USD_Spread'], lags=30, ax=axes[0, 1], color='#2196F3', alpha=0.05)
axes[0, 1].set_title('PACF — USD Spread', fontweight='bold')

plot_acf(spreads['KHR_Spread'], lags=30, ax=axes[1, 0], color='#E91E63', alpha=0.05)
axes[1, 0].set_title('ACF — KHR Spread', fontweight='bold')

plot_pacf(spreads['KHR_Spread'], lags=30, ax=axes[1, 1], color='#E91E63', alpha=0.05)
axes[1, 1].set_title('PACF — KHR Spread', fontweight='bold')

for ax in axes.flat:
    ax.set_xlabel('Lag (months)')

plt.tight_layout()
plt.savefig('../figures/fig_acf_pacf.png', dpi=300, bbox_inches='tight')
plt.show()

print('\nInterpretation: Significant ACF at lag 1 with rapid decay supports')
print('the AR(1) / mean-reverting OU process specification.')
print(f'\nAR(1) autocorrelation at lag 1:')
print(f'  USD Spread: {spreads["USD_Spread"].autocorr(lag=1):.4f}')
print(f'  KHR Spread: {spreads["KHR_Spread"].autocorr(lag=1):.4f}')

---
## 5. Correlation Analysis

In [ ]:
# ─── Correlation Analysis ────────────────────────────────────────────────────
full_corr = spreads['USD_Spread'].corr(spreads['KHR_Spread'])
print(f'Full-sample Pearson correlation: {full_corr:.4f}')
print(f'\nSub-period correlations:')
for pname, (start, end) in periods.items():
    mask = (spreads.index >= start) & (spreads.index <= end)
    sub = spreads[mask]
    corr = sub['USD_Spread'].corr(sub['KHR_Spread'])
    print(f'  {pname}: {corr:.4f}')

---
## 6. Publication-Quality Visualizations

### Figure 1: Dual Time Series with COVID Shading

In [ ]:
# ─── FIGURE 1: Dual Time Series ──────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(spreads.index, spreads['USD_Spread'], color='#1565C0', linewidth=1.5,
        label='USD Spread', alpha=0.9)
ax.plot(spreads.index, spreads['KHR_Spread'], color='#C62828', linewidth=1.5,
        label='KHR Spread', alpha=0.9)

# COVID shading
ax.axvspan(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-12-31'),
           alpha=0.12, color='grey', label='COVID-19 Period')

# Event annotations
events = [
    ('2020-03-01', 'COVID-19\nOnset', 15),
    ('2022-03-01', 'Fed Rate\nHikes Begin', 18),
    ('2023-07-01', 'Fed Peak\nRate', 15),
    ('2024-09-01', 'Fed Rate\nCuts Begin', 18),
]
for date, label, ypos in events:
    ax.annotate(label, xy=(pd.Timestamp(date), ypos),
                fontsize=7.5, ha='center', va='bottom',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow',
                          edgecolor='grey', alpha=0.8),
                arrowprops=dict(arrowstyle='->', color='grey', lw=0.8))

ax.set_xlabel('Date')
ax.set_ylabel('Interest Rate Spread (%)')
ax.set_title('Figure 1: USD and KHR Interest Rate Spreads (Jan 2013 – Dec 2025)',
             fontweight='bold', fontsize=13)
ax.legend(loc='upper right', framealpha=0.9)
ax.grid(True, alpha=0.3)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('../figures/fig1_spread_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: fig1_spread_timeseries.png')

### Figure 2: Spread Histograms with Normal Overlay

In [ ]:
# ─── FIGURE 2: Histograms ────────────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for ax, col, color, label in [(ax1, 'USD_Spread', '#1565C0', 'USD'),
                               (ax2, 'KHR_Spread', '#C62828', 'KHR')]:
    data = spreads[col]
    ax.hist(data, bins=25, density=True, alpha=0.6, color=color, edgecolor='white')
    
    # Normal overlay
    x = np.linspace(data.min() - 1, data.max() + 1, 200)
    ax.plot(x, stats.norm.pdf(x, data.mean(), data.std()),
            color='black', linewidth=1.5, linestyle='--', label='Normal fit')
    
    ax.set_xlabel(f'{label} Spread (%)')
    ax.set_ylabel('Density')
    ax.set_title(f'{label} Spread Distribution', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Stats annotation
    textstr = f'μ = {data.mean():.2f}%\nσ = {data.std():.2f}%\nSkew = {data.skew():.2f}\nKurt = {data.kurtosis():.2f}'
    ax.text(0.97, 0.97, textstr, transform=ax.transAxes, fontsize=9,
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

fig.suptitle('Figure 2: Distribution of Interest Rate Spreads', fontweight='bold', fontsize=13, y=1.02)
plt.tight_layout()
plt.savefig('../figures/fig2_spread_histograms.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: fig2_spread_histograms.png')

### Figure 3: Raw Rates (Underlying Components)

In [ ]:
# ─── FIGURE 3: Raw Rates ─────────────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 9), sharex=True)

# USD rates
ax1.plot(rates.index, rates['USD_Term_Loans'], color='#1565C0', linewidth=1.3,
         label='USD Term Loan Rate')
ax1.plot(rates.index, rates['USD_Term_Deposits'], color='#64B5F6', linewidth=1.3,
         label='USD Term Deposit Rate', linestyle='--')
ax1.fill_between(rates.index, rates['USD_Term_Deposits'], rates['USD_Term_Loans'],
                 alpha=0.1, color='#1565C0')
ax1.axvspan(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-12-31'),
            alpha=0.1, color='grey')
ax1.set_ylabel('Rate (%)')
ax1.set_title('USD — Term Loan and Term Deposit Rates', fontweight='bold')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# KHR rates
ax2.plot(rates.index, rates['KHR_Term_Loans'], color='#C62828', linewidth=1.3,
         label='KHR Term Loan Rate')
ax2.plot(rates.index, rates['KHR_Term_Deposits'], color='#EF9A9A', linewidth=1.3,
         label='KHR Term Deposit Rate', linestyle='--')
ax2.fill_between(rates.index, rates['KHR_Term_Deposits'], rates['KHR_Term_Loans'],
                 alpha=0.1, color='#C62828')
ax2.axvspan(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-12-31'),
            alpha=0.1, color='grey')
ax2.set_xlabel('Date')
ax2.set_ylabel('Rate (%)')
ax2.set_title('KHR — Term Loan and Term Deposit Rates', fontweight='bold')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

ax2.xaxis.set_major_locator(mdates.YearLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)

fig.suptitle('Figure 3: Underlying Interest Rates — New Amount (Jan 2013 – Dec 2025)',
             fontweight='bold', fontsize=13, y=1.01)
plt.tight_layout()
plt.savefig('../figures/fig3_raw_rates.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: fig3_raw_rates.png')

### Figure 4: Scatter Plot — USD vs KHR Spread

In [ ]:
# ─── FIGURE 4: Scatter Plot ──────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(8, 7))

# Color by period
colors = []
labels_done = {'Pre-COVID': False, 'COVID': False, 'Post-COVID': False}
for d in spreads.index:
    if d < pd.Timestamp('2020-01-01'):
        colors.append('#1565C0')
    elif d < pd.Timestamp('2022-01-01'):
        colors.append('#FF6F00')
    else:
        colors.append('#2E7D32')

scatter = ax.scatter(spreads['USD_Spread'], spreads['KHR_Spread'],
                     c=colors, alpha=0.6, s=40, edgecolors='white', linewidth=0.5)

# Regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(
    spreads['USD_Spread'], spreads['KHR_Spread'])
x_line = np.linspace(spreads['USD_Spread'].min(), spreads['USD_Spread'].max(), 100)
ax.plot(x_line, slope * x_line + intercept, 'k--', linewidth=1.2, alpha=0.7)

# Legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#1565C0', markersize=8, label='Pre-COVID'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#FF6F00', markersize=8, label='COVID'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#2E7D32', markersize=8, label='Post-COVID'),
]
ax.legend(handles=legend_elements, loc='upper right')

ax.set_xlabel('USD Spread (%)')
ax.set_ylabel('KHR Spread (%)')
ax.set_title('Figure 4: USD vs. KHR Spread Correlation', fontweight='bold', fontsize=13)
ax.text(0.05, 0.95, f'ρ = {r_value:.4f}\np < 0.0001',
        transform=ax.transAxes, fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../figures/fig4_correlation.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: fig4_correlation.png')

### Figure 5: Box Plots by Year

In [ ]:
# ─── FIGURE 5: Box Plots by Year ─────────────────────────────────────────────
spreads_year = spreads.copy()
spreads_year['Year'] = spreads_year.index.year

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

years = sorted(spreads_year['Year'].unique())
usd_by_year = [spreads_year[spreads_year['Year']==y]['USD_Spread'].values for y in years]
khr_by_year = [spreads_year[spreads_year['Year']==y]['KHR_Spread'].values for y in years]

bp1 = ax1.boxplot(usd_by_year, labels=years, patch_artist=True)
for box in bp1['boxes']:
    box.set(facecolor='#BBDEFB', edgecolor='#1565C0')
ax1.set_ylabel('Spread (%)')
ax1.set_title('USD Spread Distribution by Year', fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Shade COVID years
for ax in [ax1, ax2]:
    # years list is 0-indexed: 2020 is at index 7 (2013=0), 2021 at index 8
    covid_start_idx = years.index(2020)
    ax.axvspan(covid_start_idx + 0.5, covid_start_idx + 2.5, alpha=0.1, color='grey')

bp2 = ax2.boxplot(khr_by_year, labels=years, patch_artist=True)
for box in bp2['boxes']:
    box.set(facecolor='#FFCDD2', edgecolor='#C62828')
ax2.set_xlabel('Year')
ax2.set_ylabel('Spread (%)')
ax2.set_title('KHR Spread Distribution by Year', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45)

fig.suptitle('Figure 5: Annual Spread Distributions', fontweight='bold', fontsize=13, y=1.01)
plt.tight_layout()
plt.savefig('../figures/fig5_boxplots_by_year.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: fig5_boxplots_by_year.png')

---
## Summary of Key Findings

The exploratory analysis reveals:

1. **KHR spread compression**: The KHR spread has declined dramatically from ~24% in 2013 to ~4–5% by 2025, reflecting the maturation of Cambodia's financial sector and de-dollarization progress.

2. **Higher KHR volatility**: KHR spread standard deviation is ~3.5x higher than USD, indicating the riel segment carries substantially more risk instability.

3. **Stationarity**: Both spreads should exhibit stationarity (ADF test), supporting the mean-reverting OU model specification.

4. **Strong autocorrelation**: High lag-1 autocorrelation in both series is consistent with a persistent but mean-reverting process (AR(1)/OU).

5. **High cross-currency correlation**: The full-sample correlation is high (~0.84), suggesting common macro factors drive both segments, but the relationship may vary across sub-periods.

These properties validate the choice of the Ornstein-Uhlenbeck stochastic process for modeling.